In [ ]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import vectice
from vectice import FileDataWrapper


In [ ]:
# Load dataset
url = "https://raw.githubusercontent.com/vectice/GettingStarted/main/22.4/tutorial/Data/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = read_csv(url, names=names)

In [ ]:
## Goal of this sample is to simply catalog assets in Vectice

# Connect to Vectice
conn = vectice.connect(
    api_token='oB5OEdKOg.Gn9QE8yRGW7mN4jxd0ZaAoB5OEdKOgLz3P6rlk2bVDwXYMpevJ', 
    host='https://app.vectice.com',
    workspace='Samples'
)
# print(conn.projects) # get list of projects
project = conn.project(6720)

In [ ]:
# Set the origin dataset for the project
project.origin_dataset = FileDataWrapper(path= 'iris.csv', name="Iris Dataset",capture_code=False)

In [ ]:
# Plot the dataset
dataset.plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
boxplot = pyplot.savefig("boxplot.jpg")
pyplot.show()

# histograms
dataset.hist()
histogram = pyplot.savefig("histogram.jpg")
pyplot.show()

### NO WAY FOR ME TO PUSH THESE ATTACHMENTS? 23.1

In [ ]:
# Split-out validation dataset
array = dataset.values
X = array[:,0:4]
y = array[:,4]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

# Build Vectice objs
training_wrapped = FileDataWrapper(path="X_train", name="X_train", usage=vectice.DatasetSourceUsage.TRAINING, capture_code=False)
validation_wrapped = FileDataWrapper(path="X_validation", name="X_validation", usage=vectice.DatasetSourceUsage.VALIDATION, capture_code=False)

project.training_dataset = training_wrapped
project.validation_dataset = validation_wrapped

## Nothing happen

In [ ]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
# Start Vectice iteration
iterStep = project.phase("Modeling").iteration.step("Build model")
for name, model in models:
 kfold = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
 cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
 results.append(cv_results)
 names.append(name)
 metrics = {"Mean": cv_results.mean(), "Std.": cv_results.std()}
# Document model in Vectice - 1 line
 project.phase("Modeling").iterations[0].model = vectice.Model(name= 'Iris Classification Models', library= name, technique=name, metrics=metrics, attachments=['boxplot.jpg','histogram.jpg'], capture_code= False)
 #print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
# Close Vectice step
iterStep.close(message="Iteration done")